In [1]:
import tensorflow as tf
import numpy as np
from datetime import datetime

In [2]:
# MNIST dataset parameters.
num_classes = 10 # total classes (0-9 digits).
epochs = 3
batch_size = 64

In [3]:
# Prepare MNIST data.
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# Convert to float32.
x_train, x_test = np.array(x_train, np.float32), np.array(x_test, np.float32)
# Normalize images value from [0, 255] to [0, 1].
x_train, x_test = x_train / 255., x_test / 255.

# Use tf.data API to shuffle and batch data.
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat(epochs).shuffle(5000).batch(batch_size).prefetch(1)

---

In [4]:
# Training parameters.
learning_rate = 0.1

# Network parameters.
n_hidden_1 = 128  # 1st layer number of neurons.
n_hidden_2 = 256  # 2nd layer number of neurons.


# Create TF Model.
class DenseNet(tf.keras.Model):
    # Set layers.
    def __init__(self):
        super(DenseNet, self).__init__()
        self.flatten = tf.keras.layers.Flatten(input_shape=(28, 28))
        # First fully-connected hidden layer.
        self.fc1 = tf.keras.layers.Dense(n_hidden_1, activation=tf.nn.relu)
        # First fully-connected hidden layer.
        self.fc2 = tf.keras.layers.Dense(n_hidden_2, activation=tf.nn.relu)
        # Second fully-connecter hidden layer.
        self.out = tf.keras.layers.Dense(num_classes)

    # Set forward pass.
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        return tf.nn.softmax(x)


dense_model = DenseNet()

In [5]:
checkpoint_path = 'checkpoints/dense' + datetime.now().strftime(
    "%Y%m%d-%H%M%S") + '.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                verbose=1)


%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/dense/
logdir="logs/dense/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


optimizer = tf.keras.optimizers.SGD(learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_acc = tf.keras.metrics.SparseCategoricalAccuracy()

In [6]:
dense_model.compile(optimizer, loss, metrics=[metric_acc])
dense_model.fit(
    train_data,
    validation_data=(x_test, y_test),
    epochs=epochs,
    callbacks=[checkpoint, tensorboard_callback],
)

Epoch 1/3
2773/2813 [============================>.] - ETA: 0s - loss: 1.6539 - sparse_categorical_accuracy: 0.8277
Epoch 00001: val_loss improved from inf to 1.53712, saving model to checkpoints/dense20200623-190813.ckpt
2813/2813 [==============================] - 4s 1ms/step - loss: 1.6522 - sparse_categorical_accuracy: 0.8292 - val_loss: 1.5371 - val_sparse_categorical_accuracy: 0.9306
Epoch 2/3
2807/2813 [============================>.] - ETA: 0s - loss: 1.5267 - sparse_categorical_accuracy: 0.9406
Epoch 00002: val_loss improved from 1.53712 to 1.52021, saving model to checkpoints/dense20200623-190813.ckpt
2813/2813 [==============================] - 3s 1ms/step - loss: 1.5267 - sparse_categorical_accuracy: 0.9407 - val_loss: 1.5202 - val_sparse_categorical_accuracy: 0.9459
Epoch 3/3
2809/2813 [============================>.] - ETA: 0s - loss: 1.5111 - sparse_categorical_accuracy: 0.9547
Epoch 00003: val_loss improved from 1.52021 to 1.50924, saving model to checkpoints/dense20200

In [8]:
dense_model.summary()

Model: "dense_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  100480    
_________________________________________________________________
dense_1 (Dense)              multiple                  33024     
_________________________________________________________________
dense_2 (Dense)              multiple                  2570      
Total params: 136,074
Trainable params: 136,074
Non-trainable params: 0
_________________________________________________________________


In [9]:
dense_model.load_weights(checkpoint_path)
dense_model.evaluate(x_test, y_test)

313/313 [==============================] - 0s 783us/step - loss: 1.5092 - sparse_categorical_accuracy: 0.9556


[1.5092374086380005, 0.9556000232696533]

---

In [10]:
# Training parameters.
learning_rate = 0.001

# Network parameters.
conv1_filters = 32  # number of filters for 1st conv layer.
conv2_filters = 64  # number of filters for 2nd conv layer.
fc1_units = 1024  # number of neurons for 1st fully-connected layer.


# Create TF Model.
class ConvNet(tf.keras.Model):
    # Set layers.
    def __init__(self):
        super(ConvNet, self).__init__()
        # Convolution Layer with 32 filters and a kernel size of 5.
        self.conv1 = tf.keras.layers.Conv2D(32,
                                            kernel_size=5,
                                            activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2.
        self.maxpool1 = tf.keras.layers.MaxPool2D(2, strides=2)

        # Convolution Layer with 64 filters and a kernel size of 3.
        self.conv2 = tf.keras.layers.Conv2D(64,
                                            kernel_size=3,
                                            activation=tf.nn.relu)
        # Max Pooling (down-sampling) with kernel size of 2 and strides of 2.
        self.maxpool2 = tf.keras.layers.MaxPool2D(2, strides=2)

        # Flatten the data to a 1-D vector for the fully connected layer.
        self.flatten = tf.keras.layers.Flatten()

        # Fully connected layer.
        self.fc1 = tf.keras.layers.Dense(1024)
        # Apply Dropout (if is_training is False, dropout is not applied).
        self.dropout = tf.keras.layers.Dropout(rate=0.5)

        # Output layer, class prediction.
        self.out = tf.keras.layers.Dense(num_classes)

    # Set forward pass.
    def call(self, inputs):
        x = tf.reshape(inputs, [-1, 28, 28, 1])
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.out(x)
        return tf.nn.softmax(x)


conv_model = ConvNet()

In [11]:
checkpoint_path = 'checkpoints/conv' + datetime.now().strftime(
    "%Y%m%d-%H%M%S") + '.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                verbose=1)


%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/conv/
logdir="logs/conv/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


optimizer = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_acc = tf.keras.metrics.SparseCategoricalAccuracy()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [12]:
conv_model.compile(optimizer, loss, metrics=[metric_acc])
conv_model.fit(
    train_data,
    validation_data=(x_test, y_test),
    epochs=epochs,
    callbacks=[checkpoint, tensorboard_callback],
)

Epoch 1/3
2813/2813 [==============================] - ETA: 0s - loss: 1.5012 - sparse_categorical_accuracy: 0.9606
Epoch 00001: val_loss improved from inf to 1.47929, saving model to checkpoints/conv20200623-190825.ckpt
2813/2813 [==============================] - 86s 30ms/step - loss: 1.5012 - sparse_categorical_accuracy: 0.9606 - val_loss: 1.4793 - val_sparse_categorical_accuracy: 0.9817
Epoch 2/3
2812/2813 [============================>.] - ETA: 0s - loss: 1.4819 - sparse_categorical_accuracy: 0.9791
Epoch 00002: val_loss did not improve from 1.47929
2813/2813 [==============================] - 103s 37ms/step - loss: 1.4819 - sparse_categorical_accuracy: 0.9791 - val_loss: 1.4823 - val_sparse_categorical_accuracy: 0.9787
Epoch 3/3
2813/2813 [==============================] - ETA: 0s - loss: 1.4814 - sparse_categorical_accuracy: 0.9797
Epoch 00003: val_loss did not improve from 1.47929
2813/2813 [==============================] - 86s 31ms/step - loss: 1.4814 - sparse_categorical_acc

In [14]:
conv_model.summary()

Model: "conv_net"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  832       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
conv2d_1 (Conv2D)            multiple                  18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 multiple                  0         
_________________________________________________________________
flatten_1 (Flatten)          multiple                  0         
_________________________________________________________________
dense_3 (Dense)              multiple                  1639424   
_________________________________________________________________
dropout (Dropout)            multiple                  0  

In [15]:
conv_model.load_weights(checkpoint_path)
conv_model.evaluate(x_test, y_test)

313/313 [==============================] - 2s 7ms/step - loss: 1.4793 - sparse_categorical_accuracy: 0.9817


[1.4792938232421875, 0.9817000031471252]

---

In [16]:
# Training Parameters
learning_rate = 0.001


# Network Parameters
# MNIST image shape is 28*28px, we will then handle 28 sequences of 28 timesteps for every sample.
num_input = 28  # number of sequences.
timesteps = 28  # timesteps.
num_units = 32  # number of neurons for the LSTM layer.


# Create LSTM Model.
class LSTM(tf.keras.Model):
    # Set layers.
    def __init__(self):
        super(LSTM, self).__init__()
        self.reshape = tf.keras.layers.Lambda(lambda x: tf.reshape(x, [-1, 28, 28]))
        # RNN (LSTM) hidden layer.
        self.lstm_layer = tf.keras.layers.LSTM(units=num_units)
        self.out = tf.keras.layers.Dense(num_classes)

    # Set forward pass.
    def call(self, inputs):
        # LSTM layer.
        x = self.reshape(inputs)
        x = self.lstm_layer(x)
        # Output layer (num_classes).
        x = self.out(x)
        return tf.nn.softmax(x)

lstm_model = LSTM()

In [17]:
checkpoint_path = 'checkpoints/lstm' + datetime.now().strftime(
    "%Y%m%d-%H%M%S") + '.ckpt'
checkpoint = tf.keras.callbacks.ModelCheckpoint(checkpoint_path,
                                                save_best_only=True,
                                                save_weights_only=True,
                                                monitor='val_loss',
                                                verbose=1)


%load_ext tensorboard
# Clear any logs from previous runs
!rm -rf ./logs/lstm/
logdir="logs/lstm/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)


optimizer = tf.keras.optimizers.Adam(learning_rate)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric_acc = tf.keras.metrics.SparseCategoricalAccuracy()

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [18]:
lstm_model.compile(optimizer, loss, metrics=[metric_acc])
lstm_model.fit(
    train_data,
    validation_data=(x_test, y_test),
    epochs=epochs,
    callbacks=[checkpoint, tensorboard_callback],
)

Epoch 1/3
2810/2813 [============================>.] - ETA: 0s - loss: 1.6393 - sparse_categorical_accuracy: 0.8377
Epoch 00001: val_loss improved from inf to 1.53038, saving model to checkpoints/lstm20200623-191303.ckpt
2813/2813 [==============================] - 29s 10ms/step - loss: 1.6393 - sparse_categorical_accuracy: 0.8377 - val_loss: 1.5304 - val_sparse_categorical_accuracy: 0.9352
Epoch 2/3
2810/2813 [============================>.] - ETA: 0s - loss: 1.5207 - sparse_categorical_accuracy: 0.9437
Epoch 00002: val_loss improved from 1.53038 to 1.51104, saving model to checkpoints/lstm20200623-191303.ckpt
2813/2813 [==============================] - 29s 10ms/step - loss: 1.5207 - sparse_categorical_accuracy: 0.9436 - val_loss: 1.5110 - val_sparse_categorical_accuracy: 0.9517
Epoch 3/3
2812/2813 [============================>.] - ETA: 0s - loss: 1.5044 - sparse_categorical_accuracy: 0.9586
Epoch 00003: val_loss improved from 1.51104 to 1.50151, saving model to checkpoints/lstm2020

In [20]:
lstm_model.summary()

Model: "lstm"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda (Lambda)              multiple                  0         
_________________________________________________________________
lstm_1 (LSTM)                multiple                  7808      
_________________________________________________________________
dense_5 (Dense)              multiple                  330       
Total params: 8,138
Trainable params: 8,138
Non-trainable params: 0
_________________________________________________________________


In [21]:
lstm_model.load_weights(checkpoint_path)
lstm_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 1.5015 - sparse_categorical_accuracy: 0.9603


[1.5015079975128174, 0.9603000283241272]

---

In [22]:
%tensorboard --logdir logs

Reusing TensorBoard on port 6008 (pid 61684), started 1:54:52 ago. (Use '!kill 61684' to kill it.)